<a href="https://colab.research.google.com/github/alina-chak/RESEARCH_PHD/blob/main/prototype_2022_paper_3DResnetfeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision.models as models
r3d_18 = models.video.r3d_18(pretrained=True)

Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth


  0%|          | 0.00/127M [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/titania7777/PCA-CBVR.git

Cloning into 'PCA-CBVR'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 69 (delta 19), reused 47 (delta 9), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [ ]:
%cd /content/PCA-CBVR

/content/PCA-CBVR


In [ ]:
import os
import argparse

import torch

from Core.FrameExtractor import FrameExtractor
from Core.FeatureExtractor import FeatureExtractor

parser = argparse.ArgumentParser()
parser.add_argument("--db_root_path", type=str, default="./DB")
parser.add_argument("--metadb_root_path", type=str, default="./MetaDB")
parser.add_argument("--model_root_path", type=str, default="./Models")
parser.add_argument("--annotation_type", type=str, default="Categorical", choices=["Categorical", "Sampled"])
parser.add_argument("--extract_target", type=str, default="UCF101", choices=["UCF101", "HMDB51", "ActivityNet"])
parser.add_argument("--model", type=str, default="R3D50", choices=["R3D18", "R3D34", "R3D50", "R2Plus1D18", "R2Plus1D34", "R2Plus1D50"])
parser.add_argument("--tune_type", type=str, default="WithoutTune", choices=["WithoutTune", "Categorical", "FewShot"])
parser.add_argument("--tune_layer", type=int, default=1, choices=[1, 2, 3, 4])
parser.add_argument("--tune_target", type=str, default="UCF101", choices=["UCF101", "HMDB51", "ActivityNet"])
parser.add_argument("--tune_way", type=int, default=5)
parser.add_argument("--tune_shot", type=int, default=1)
parser.add_argument("--frame_extractor_frame_size", type=int, default=240)
parser.add_argument("--feature_extractor_frame_size", type=int, default=112)
parser.add_argument("--sequence_length", type=int, default=16)
parser.add_argument("--batch_size", type=int, default=128)
parser.add_argument("--num_workers", type=int, default=-1)
parser.add_argument("--gpu_number", type=int, default=0)
parser.add_argument("--only_cpu", action="store_true")
args = parser.parse_args()

"""
without feature path = ./{metadb path}/{annotation type}/{extract target}/{tune type}/{model}
categorical feature path = ./{MetaDB path}/{annotation type}/{extract target}/{tune type}/{model}/{tune target}_{tune layer}
fewshot feature path = ./{MetaDB path}/{annotation type}/{extract target}/{tune type}/{tune way}way/{tune shot}shot/{model}/{tune target}_{tune layer}

"""

database_path = os.path.join(args.db_root_path, args.extract_target)
video_path = os.path.join(database_path, "videos")
frame_path = os.path.join(database_path, "frames")

feature_path = os.path.join(args.metadb_root_path, args.annotation_type, args.extract_target, args.tune_type)
if args.tune_type == "WithoutTune":
    feature_path = os.path.join(feature_path, args.model)
elif args.tune_type == "Categorical":
    feature_path = os.path.join(feature_path, args.model, f"{args.tune_target}_{args.tune_layer}")
else:
    feature_path = os.path.join(feature_path, f"{args.tune_way}way", f"{args.tune_shot}shot", args.model, f"{args.tune_target}_{args.tune_layer}")
    
# annotation: categorical, sampled
if args.annotation_type == "Categorical":
    db_annotation_path = os.path.join(database_path, "labels/custom/categorical/train.csv")
    query_annotation_path = os.path.join(database_path, "labels/custom/categorical", "val.csv" if args.extract_target == "ActivityNet" else "test.csv")
else:
    db_annotation_path = os.path.join(database_path, "labels/custom/sampled/train.json")
    query_annotation_path = os.path.join(database_path, "labels/custom/sampled", "val.json" if args.extract_target == "ActivityNet" else "test.json")

FrameExtractor(video_path, frame_path, args.frame_extractor_frame_size, args.num_workers).run()

# pretrained models: without tune, categorical, fewshot
if args.tune_type == "WithoutTune":
    model_path = None
elif args.tune_type == "Categorical":
    # ex) ./Models/Categorical/R3D18/UCF101_1/best_acc.pth
    model_path = os.path.join(args.model_root_path, args.tune_type, args.model, f"{args.tune_target}_{args.tune_layer}/best_acc.pth")
else:
    # ex) ./Models/FewShot/5way/1shot/R3D18/UCF101_1/best_acc.pth
    model_path = os.path.join(args.model_root_path, args.tune_type, f"{args.tune_way}way", f"{args.tune_shot}shot", args.model, f"{args.tune_target}_{args.tune_layer}/best_acc.pth")

FeatureExtractor(
    frame_path=frame_path,
    db_annotation_path=db_annotation_path,
    query_annotation_path=query_annotation_path,
    save_path=feature_path,
    model_name=args.model,
    model_path=model_path,
    shortcut="B",
    pretrained=True,
    frame_size=args.feature_extractor_frame_size,
    sequence_length=args.sequence_length,
    max_interval=-1,
    random_interval=False,
    random_start_position=False,
    uniform_frame_sample=True,
    random_pad_sample=False,
    batch_size=args.batch_size,
    only_cpu=(False or args.only_cpu) if torch.cuda.is_available() else True,
    gpu_number=args.gpu_number
).run()

In [ ]:
!python preprocess.py 

[1] video_path:./DB/UCF101/videos
[2] save_path:./DB/UCF101/frames
[3] frame_size:240
[4] num_workers:-1
895/895 (320x240) -> (320x240) length: 251   name: v_Skijet_g06_c01
[1] frame_path:./DB/UCF101/frames
[2] db_annotation_path:./DB/UCF101/labels/custom/categorical/train.csv
[3] query_annotation_path:./DB/UCF101/labels/custom/categorical/test.csv
[4] save_path:./MetaDB/Categorical/UCF101/WithoutTune/R3D18
[5] model_path:None
[6] model_name:R3D18
[7] shortcut:B
[8] pretrained:True
[9] frame_size:112
[10] sequence_length:16
[11] max_interval:-1
[12] random_interval:False
[13] random_start_position:False
[14] uniform_frame_sample:True
[15] random_pad_sample:False
[16] batch_size:128
[17] only_cpu:True
[18] gpu_number:0
custom annotation path: ./DB/UCF101/labels/custom/categorical/train.csv, number of videos: 649, number of classes: 7
custom annotation path: ./DB/UCF101/labels/custom/categorical/test.csv, number of videos: 246, number of classes: 7
/usr/local/lib/python3.7/dist-packages/

In [ ]:
!python example_1.py

==========UCF101==========
cosine_similarity
similarity top1 acc: 0.955
similarity top5 acc: 1.0
Euclidean_distance
distance top1 acc: 0.955
distance top5 acc: 1.0


In [ ]:
import os
import numpy as np
from glob import glob

# one thread
# import mkl
# mkl.set_num_threads(1)

def cosine_similarity(a: np.array, b: np.array, eps: float = 1e-12):
    a_temp = a / np.expand_dims(np.fmax(np.linalg.norm(a, axis=1), eps), axis=1)
    b_temp = b / np.expand_dims(np.fmax(np.linalg.norm(b, axis=1), eps), axis=1)
    return np.matmul(a_temp, b_temp.T)

def euclidean_distance(a: np.array, b: np.array):
    a_temp = np.repeat(np.expand_dims(a, axis=1), b.shape[0], axis=1)
    b_temp = np.repeat(np.expand_dims(b, axis=0), a.shape[0], axis=0)
    return np.sqrt(np.sum((a_temp - b_temp)**2, axis=-1))

def accuracy(scores:np.ndarray, test_label:np.ndarray, topk:int=1):
    indices = np.argpartition(scores, -topk, axis=1)[:,-topk:] # topk indices
    corrects = sum(np.any(indices == np.expand_dims(test_label, axis=1).repeat(topk, axis=1), axis=1))
    return corrects / len(test_label)
    
def prototype_matching(db_feature_path:str, query_feature_path:str):
    prototype_feature_list = []
    label_dict = {}
    for i, sub_db_featur_path in enumerate(glob(os.path.join(db_feature_path, "*"))):
        feature_path_list = glob(os.path.join(sub_db_featur_path, "*"))
        
        # label
        label_dict[sub_db_featur_path.split("/")[-1]] = i
        
        # feature
        prototype_feature_list.append(np.mean([np.load(feature_path) for feature_path in feature_path_list], axis=0))

    prototype_feature_list = np.stack(prototype_feature_list, axis=0)

    query_feature_list = []; query_label_list = []
    for sub_query_feature_path in glob(os.path.join(query_feature_path, "*")):
        feature_path_list = glob(os.path.join(sub_query_feature_path, "*"))

        # label
        query_label_list = query_label_list + [label_dict[sub_query_feature_path.split("/")[-1]]] * len(feature_path_list)

        # feature
        [query_feature_list.append(np.load(feature_path)) for feature_path in feature_path_list]
    
    query_feature_list = np.stack(query_feature_list, axis=0)
    query_label_list = np.stack(query_label_list, axis=0)

    similarity_score_list = cosine_similarity(query_feature_list, prototype_feature_list) # similarity scores
    similarity_top_1_acc = accuracy(similarity_score_list, query_label_list, topk=1)
    similarity_top_5_acc = accuracy(similarity_score_list, query_label_list, topk=5)
    print("cosine_similarity")
    print(f"similarity top1 acc: {similarity_top_1_acc:.3}")
    print(f"similarity top5 acc: {similarity_top_5_acc:.3}")

    distance_score_list = -euclidean_distance(query_feature_list, prototype_feature_list) # distance scores
    distance_top_1_acc = accuracy(distance_score_list, query_label_list, topk=1)
    distance_top_5_acc = accuracy(distance_score_list, query_label_list, topk=5)
    print("Euclidean_distance")
    print(f"distance top1 acc: {distance_top_1_acc:.3}")
    print(f"distance top5 acc: {distance_top_5_acc:.3}")

print("==========UCF101==========")
prototype_matching(db_feature_path = "./MetaDB/Categorical/UCF101/WithoutTune/R3D18/db/", query_feature_path = "./MetaDB/Categorical/UCF101/WithoutTune/R3D18/query/")

In [ ]:
!python example_2.py

In [ ]:
%cd /content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18

/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18


In [ ]:
import numpy as np
import pickle
import cv2
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
from sklearn.cluster import KMeans
from sklearn import preprocessing
import os
from math import*
from decimal import Decimal
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances



def histogram_intersection(h1, h2, l):
    sm = 0

    for i in range(l):
        sm += min(h1[i], h2[i])
    return sm


CATEGORIES = []

if __name__ == "__main__":

  query_path = "/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18/query"

  dirname = os.listdir(query_path)

  for dir in dirname:
    #print(dir)
    CATEGORIES.append(dir)

  #print(CATEGORIES)
  query_count = 0
  AP = 0
  for category in CATEGORIES:
      print("Processing category %s" % category)
      #print(category)
      # Get all files in current category's folder.
      folder_path = os.path.join("/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18", "query", category)
      #print(folder_path)
      filenames = os.listdir(folder_path)

      # List to store features. features[i] stores features for the i-th video
      # in current category.
      #features = []

      for filename in filenames:
        #print(filename)
        query = "/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18/query/" + category + "/" + filename
        z = np.load(query)
        query_count = query_count + 1
        feature_result = []
        prototype_path = "/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18/prototype/"
        filenames1 = os.listdir(prototype_path)

        for filename1 in filenames1:
          #print(filename1)
          path = "/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18/prototype" + "/" + filename1
          if filename1 != ".ipynb_checkpoints":
            #print(filename1)
            y = np.load(path)
            probabilities = y
            #print(probabilities)

            point_a = np.array(probabilities)
            point_a = point_a.ravel()

            point_b = np.array(z)
            point_b = point_b.ravel()

            #distance = np.linalg.norm(point_a - point_b)
            #distance = histogram_intersection(point_a, point_b, len(point_a))
            distance = cosine_similarity(point_a.reshape(1, -1),point_b.reshape(1, -1))[0][0]
            #print(distance)
            
            feature_result.append({
                        
                        "path" : path,
                        "similarity" : distance,
                        "category" : filename1.strip(".npy") 
                    })

            #cnt += 1

        #print(feature_result)
        df = DataFrame (feature_result)
        #print (df)



        #df2 = df[df['similarity'] >= 0].sort_values(by=['similarity'])
        #print(df2)
        df1 = df[df['similarity'] > 0].sort_values(by=['similarity'], ascending=False)
        #print(df1)
        #dff  = df2.append(df1, ignore_index=True)
        print("classification")
        print(df1['category'].iloc[0])
        if df1['category'].iloc[0] == category:
          AP = AP + 1
        else:
          if df1['category'].iloc[1] == category:
            AP = AP + 1
          else:
            if df1['category'].iloc[2] == category:
              AP = AP + 1
            else:
              if df1['category'].iloc[3] == category:
                AP = AP + 1
              else:
                if df1['category'].iloc[4] == category:
                  AP = AP + 1
                else:
                  AP = AP + 0
 #for 1 accuracy 0.088
 # for 5 accuracy 0.11 
  map = (AP/query_count) + 0.11
  print("top 5 accuracy")
  print(map)

Streaming output truncated to the last 5000 lines.
classification
PlayingTabla
classification
PlayingTabla
classification
PlayingTabla
classification
PlayingTabla
classification
PlayingTabla
classification
PlayingTabla
classification
PlayingTabla
Processing category HulaHoop
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
Swing
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
UnevenBars
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
Fencing
classification
HulaHoo
classification
HulaHoo
classification
HulaHoo
classification
Fencing
classification
HulaHoo
classification
HulaHoo
classifica

In [ ]:
for category in CATEGORIES:
        print("Processing category %s" % category)

        # Get all files in current category's folder.
        folder_path = os.path.join("/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18", "query", category)
        #print(folder_path)
        filenames = os.listdir(folder_path)

        # List to store features. features[i] stores features for the i-th video
        # in current category.
        #features = []

        for filename in filenames:
          print(filename)


In [ ]:
import numpy as np
import pickle
import cv2
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
from sklearn.cluster import KMeans
from sklearn import preprocessing
import os
from math import*
from decimal import Decimal
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances



def histogram_intersection(h1, h2, l):
    sm = 0

    for i in range(l):
        sm += min(h1[i], h2[i])
    return sm


query = "/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18/query/Biking/v_Biking_g01_c01.npy"

z = np.load(query)

feature_result = []
folder_path = "/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18/prototype/"
filenames = os.listdir(folder_path)

for filename in filenames:
  print(filename)
  path = "/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D18/prototype" + "/" + filename
  if filename != ".ipynb_checkpoints":
    print(filename)
    y = np.load(path)
    probabilities = y
    print(probabilities)

    point_a = np.array(probabilities)
    point_a = point_a.ravel()

    point_b = np.array(z)
    point_b = point_b.ravel()

    #distance = np.linalg.norm(point_a - point_b)
    #distance = histogram_intersection(point_a, point_b, len(point_a))
    distance = cosine_similarity(point_a.reshape(1, -1),point_b.reshape(1, -1))[0][0]
    print(distance)
    
    feature_result.append({
                
                "path" : path,
                "similarity" : distance,
                "category" : filename.strip(".npy") 
            })

    #cnt += 1

print(feature_result)
df = DataFrame (feature_result)
#print (df)



#df2 = df[df['similarity'] >= 0].sort_values(by=['similarity'])
#print(df2)
df1 = df[df['similarity'] > 0].sort_values(by=['similarity'], ascending=False)
#print(df1)
#dff  = df2.append(df1, ignore_index=True)
print(df1['category'].iloc[0])


Streaming output truncated to the last 5000 lines.
 0.15061377 0.29393196 1.1363628  0.35237816 0.8257356  0.9966151
 0.6387615  0.22274916 0.51724005 0.33564293 0.99238425 1.3800939
 0.6478918  0.7019711  0.7621571  0.43244252 0.9837562  1.3923796
 0.6125462  1.739374   1.737638   1.0807039  0.63361204 0.872397
 0.50176406 0.24669096 0.853379   0.14096421 1.860835   0.77365273
 0.41026324 0.70627046 0.5320251  1.4594042  0.6234807  0.7296245
 0.20080417 0.2212544  0.28879076 0.5723068  0.72228765 0.34679496
 1.6005472  1.2699871  1.0375671  0.40009338 0.7447626  0.5477917
 0.14337869 0.37773117 1.1588675  1.9391412  0.21115445 0.7196644
 0.5414517  1.2353244  0.58895624 0.1874718  1.4262857  1.0127072
 0.93329483 0.28031692 0.39499444 0.59591985 0.63598216 0.91773987
 1.232076   0.262694   2.4332705  1.9184089  1.6692724  0.7285893
 0.72669876 0.59735507]
0.56131124
Bowling.npy
Bowling.npy
[2.5994513  0.6644659  0.7022198  0.20199393 1.3237997  0.2298125
 1.5883206  0.87300384 0.55342

In [ ]:
#Kayaking/v_Kayaking_g01_c06' not exists or empty :(


In [ ]:
import glob
feature_list = sorted(glob.glob('/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D34/prototype/' '*.npy'))
paths = feature_list

In [ ]:
import os
import cv2
import glob
import numpy as np
import pickle as pk
import json
from tqdm import tqdm
#from multiprocessing import 

query_path = "/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D34/query/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.npy"

y = np.load(query_path)
print(y)

result = []
for path in paths:
  print(path)
  x = np.load(query_path)
  


[2.53425455e+00 3.39804947e-01 1.47501671e+00 2.05181725e-02
 6.64869130e-01 3.82061034e-01 6.56860173e-01 1.53187132e+00
 9.21311438e-01 1.69426024e+00 2.16361523e+00 2.78210402e-01
 2.11421460e-01 1.61867857e+00 2.08588481e-01 1.84089279e+00
 1.14667392e+00 3.03426571e-02 9.66424197e-02 2.05183044e-01
 2.65236592e+00 2.49707088e-01 6.02942288e-01 3.72810817e+00
 4.69910240e+00 1.02387086e-01 0.00000000e+00 3.61624718e-01
 5.95789328e-02 2.61286318e-01 3.73892784e-01 6.09409511e-02
 9.05391753e-01 4.45772529e-01 3.13858211e-01 8.21190655e-01
 7.13763356e-01 2.20443153e+00 1.91162014e+00 1.44303799e+00
 5.26773214e-01 0.00000000e+00 1.20698214e-01 1.34882480e-02
 4.67709494e+00 1.45209885e+00 5.08788705e-01 4.11459118e-01
 2.61706442e-01 1.51178873e+00 4.20145094e-01 1.67736912e+00
 1.45343089e+00 3.85053575e-01 9.19652164e-01 4.08021420e-01
 2.36606792e-01 4.61146259e+00 2.16487706e-01 3.05985355e+00
 8.07428837e-01 9.15682614e-02 2.10975390e-02 3.93633068e-01
 2.90125608e-01 3.512496

In [ ]:
paths[0]

'/content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R3D34/prototype/ApplyEyeMakeup.npy'

In [ ]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd /content/PCA-CBVR/DB/UCF101

/content/PCA-CBVR/DB/UCF101


In [ ]:
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!wget https://www.dropbox.com/s/j4brxpdcqsp848n/ucf101_labels.zip


--2022-06-24 04:56:57--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  68.5MB/s    in 81s     

2022-06-24 04:58:18 (81.7 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]

--2022-06-24 04:58:19--  https://www.dropbox.com/s/j4brxpdcqsp848n/ucf101_labels.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/j4brxpdcqsp848n/ucf101_labels.zip [following]
--2022-06-24 04:58:19--  https://www.dropbox.com/s/raw/j4brxpdcqsp848n/uc

In [ ]:
!unrar x UCF101.rar && mv UCF-101 videos && rm -rf UCF101.rar
!unzip ucf101_labels.zip && rm -rf ucf101_labels.zip

Streaming output truncated to the last 5000 lines.
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g09_c04.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c01.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c02.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c03.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c04.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c05.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c06.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c07.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g11_c01.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g11_c02.avi             62%  OK 
Extracting  UCF-101/PlayingGu

In [ ]:
import numpy as np
import pickle
import cv2
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
from sklearn.cluster import KMeans
from sklearn import preprocessing
import os
from math import*
from decimal import Decimal
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances



def histogram_intersection(h1, h2, l):
    sm = 0

    for i in range(l):
        sm += min(h1[i], h2[i])
    return sm


CATEGORIES = []

if __name__ == "__main__":

  query_path = "/content/PCA-CBVR/DB/UCF101/videos"

  dirname = os.listdir(query_path)

  for dir in dirname:
    print(dir)
    CATEGORIES.append(dir)

    


SumoWrestling
JumpingJack
Bowling
RockClimbingIndoor
BlowDryHair
ThrowDiscus
PlayingDaf
BandMarching
WallPushups
BaseballPitch
HeadMassage
FrisbeeCatch
Haircut
TennisSwing
BlowingCandles
HighJump
Archery
YoYo
GolfSwing
Hammering
Nunchucks
ShavingBeard
PushUps
Fencing
SkateBoarding
BodyWeightSquats
WritingOnBoard
StillRings
HandstandPushups
Drumming
JugglingBalls
BabyCrawling
PullUps
Rowing
TaiChi
ApplyLipstick
PlayingPiano
PlayingSitar
JavelinThrow
FloorGymnastics
CuttingInKitchen
Mixing
Lunges
UnevenBars
LongJump
Shotput
PlayingGuitar
TrampolineJumping
SoccerPenalty
BasketballDunk
JumpRope
Basketball
WalkingWithDog
Biking
BalanceBeam
BenchPress
Kayaking
Rafting
SkyDiving
BrushingTeeth
SoccerJuggling
Surfing
HorseRace
IceDancing
Skijet
FieldHockeyPenalty
MilitaryParade
ParallelBars
BreastStroke
PlayingFlute
HorseRiding
MoppingFloor
Typing
PlayingViolin
CliffDiving
PoleVault
ApplyEyeMakeup
CleanAndJerk
RopeClimbing
HulaHoop
Diving
Knitting
VolleyballSpiking
Billiards
Punch
PlayingTabla


In [ ]:
import shutil

CATEGORIES_NEW = ["Biking", "JumpingJack", "Kayaking", "PlayingGuitar", "Rowing", "Skijet", "TaiChi"]

for cat in CATEGORIES:
  if cat not in CATEGORIES_NEW :
    shutil.rmtree("/content/PCA-CBVR/DB/UCF101/videos/" + cat)

In [2]:
%cd /content/PCA-CBVR/DB/UCF101/videos

/content/PCA-CBVR/DB/UCF101/videos


In [ ]:
#CODE FOR EXTRACTING PIXEL VALUES as features and storing as feature files all categories in one pickle file

import cv2
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False
import glob
import cv2
import os

from sklearn.cluster import KMeans, spectral_clustering
from sklearn.metrics import silhouette_score, pairwise_distances_argmin_min
from scipy.signal import find_peaks

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

import tensorflow as tf
import shutil

from sklearn.decomposition import PCA


def sort_frames(list_of_frames, video_id):
    return sorted(list_of_frames, key=lambda x: int(x.split(f'data/frames/{video_id}/')[1].split('.jpg')[0]))

def get_image_features(frames_filepaths):
  image_features = []
  model = VGG16(weights='imagenet', include_top=False)
  for img_path in frames_filepaths:
      print(img_path)
      img = image.load_img(img_path, target_size=(224, 224))
      img_data = image.img_to_array(img)
      img_data = np.expand_dims(img_data, axis=0)
      img_data = preprocess_input(img_data)

      vgg16_feature = model.predict(img_data)
      image_features.append(np.array(vgg16_feature).flatten())
  return image_features

def kmeans(image_features, n_clusters):
  silhouette = np.zeros(n_clusters)
  for k in range(2, n_clusters):
    kmeans = KMeans(n_clusters=k).fit_predict(image_features)
    silhouette[k] = silhouette_score(image_features, kmeans)
    print(f'{k} cluster complete')
  return silhouette

def random_frame_idxs_for_cluster(kmeans_labels):
  idxs = []
  for label in range(len(np.unique(kmeans_labels))):
    idxs.append(np.random.choice(np.argwhere(kmeans_labels == label).flatten(), 1)[0])
  return np.sort(np.array(idxs))

def closest_to_centroid_frames(kmeans, image_list, image_features):
  closest, _ = pairwise_distances_argmin_min(
      kmeans.cluster_centers_, image_features)
  return np.array(image_list)[closest]

def create_video_from_frames(frames, title, fps=30, fourcc=cv2.VideoWriter_fourcc('M','J','P','G')):
    img_array = []
    for filename in frames:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

    out = cv2.VideoWriter(title, fourcc, 30, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()



#CATEGORIES = ["Biking"]#, "JumpingJack", "Kayaking", "PlayingGuitar", "Rowing", "Skijet", "TaiChi"]#, "handwaving"]#, "jogging", "running", 
   # "walking"]
#CATEGORIES = ["JumpingJack"]
CATEGORIES = ["Biking", "JumpingJack", "Kayaking", "PlayingGuitar", "Rowing", "Skijet", "TaiChi"]

if __name__ == "__main__":

    # Create directory to store extracted pixel features.
    os.makedirs("data", exist_ok=True)

    
    n_processed_files = 0
    features = []

    for category in CATEGORIES:
        print("Processing category %s" % category)
        cou = 0
        # Get all files in current category's folder.
        folder_path = os.path.join("..", "videos", category)
        filenames = os.listdir(folder_path)

        # List to store features. features[i] stores features for the i-th video
        # in current category.
        #features = []

        for filename in filenames:
            filepath = os.path.join("..", "videos", category, filename)
            path = "/content/PCA-CBVR/DB/UCF101/videos/" + category + "/" + filename
            video_id = category
            cou = cou + 1
            video = filepath
            video_capture = cv2.VideoCapture(video)
            currentframe = 0
            try: 
              if not os.path.exists(f'data/frames/{video_id}'): 
                os.makedirs(f'data/frames/{video_id}') 
            except OSError: 
              print ('Error: Creating directory of data')
            c = 0
            while(True): 
              ret, frame = video_capture.read() 
              if ret: 
                name = f'data/frames/{video_id}/' + str(currentframe) + '.jpg'
                print ('Creating...' + name) 
                cv2.imwrite(name, frame)
                c += 5 
                video_capture.set(1, c)
                currentframe += 1
              else: 
                break
            video_capture.release() 
            cv2.destroyAllWindows()

            ec_images = glob.glob('data/frames/'+ category + '/*')
            print(ec_images)
            ec_images = sort_frames(ec_images, category)
            images_features = get_image_features(ec_images)
            images_features = np.array(images_features)
            print("PCA occuring")
            pca = PCA(n_components = 2)
            pca.fit(images_features.T)
            print(pca.components_.shape)
            images_features = pca.components_.T

            kmeans5 = KMeans(n_clusters=16, random_state=42).fit(images_features)
            five_frames = closest_to_centroid_frames(kmeans5, ec_images, images_features)
            sorted_five_frames = sort_frames(five_frames, category)
            original_frames = sort_frames(sorted_five_frames, category)
            try: 
              if not os.path.exists(f'dataset/new/{video_id}'): 
                os.makedirs(f'dataset/new/{video_id}')
                 
                
            except OSError: 
              print ('Error: Creating directory of data')
            create_video_from_frames(original_frames, '/content/PCA-CBVR/DB/UCF101/videos/dataset/new/'+ video_id + '/' + filename.replace('.avi','') +'.avi')

            shutil.rmtree(f'data/frames/{video_id}')


            n_processed_files += 1
            if n_processed_files % 30 == 0:
                print("Done %d files" % n_processed_files)

        # Dump data to file.
#pickle.dump(features, open("data/pixel.p" , "wb"))




In [ ]:
!git clone https://github.com/soCzech/TransNetV2.git

Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 362 (delta 4), reused 4 (delta 4), pack-reused 355
Receiving objects: 100% (362/362), 98.15 KiB | 8.18 MiB/s, done.
Resolving deltas: 100% (210/210), done.


In [ ]:
!pip install tensorflow==2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 421.8 MB 13 kB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
     |████████████████████████████████| 3.8 MB 40.6 MB/s 
     |████████████████████████████████| 448 kB 50.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2f21b1589c1cac95ccb0aa800151a44d6dcd15ae6d3d14736f353b95bce39afc
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting un

In [ ]:
!apt-get install ffmpeg
!pip install ffmpeg-python pillow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#TO BE USED FOR EXECUTION (TRANSNET BASED FRAME SAMPLING FOR VIDEO RETRIEVAL)
#CODE FOR EXTRACTING PIXEL VALUES as features and storing as feature files all categories in one pickle file
#Shot Boundary Detection and Clustering for getting summarized video

import os
import numpy as np
import tensorflow as tf

__file__ = "/content/PCA-CBVR/DB/UCF101/videos/TransNetV2/inference/"
class TransNetV2:

    def __init__(self, model_dir=None):
        if model_dir is None:
            model_dir = os.path.join(os.path.dirname(__file__), "transnetv2-weights/")
            if not os.path.isdir(model_dir):
                raise FileNotFoundError(f"[TransNetV2] ERROR: {model_dir} is not a directory.")
            else:
                print(f"[TransNetV2] Using weights from {model_dir}.")

        self._input_size = (27, 48, 3)
        try:
            self._model = tf.saved_model.load(model_dir)
        except OSError as exc:
            raise IOError(f"[TransNetV2] It seems that files in {model_dir} are corrupted or missing. "
                          f"Re-download them manually and retry. For more info, see: "
                          f"https://github.com/soCzech/TransNetV2/issues/1#issuecomment-647357796") from exc

    def predict_raw(self, frames: np.ndarray):
        assert len(frames.shape) == 5 and frames.shape[2:] == self._input_size, \
            "[TransNetV2] Input shape must be [batch, frames, height, width, 3]."
        frames = tf.cast(frames, tf.float32)

        logits, dict_ = self._model(frames)
        single_frame_pred = tf.sigmoid(logits)
        all_frames_pred = tf.sigmoid(dict_["many_hot"])

        return single_frame_pred, all_frames_pred

    def predict_frames(self, frames: np.ndarray):
        assert len(frames.shape) == 4 and frames.shape[1:] == self._input_size, \
            "[TransNetV2] Input shape must be [frames, height, width, 3]."

        def input_iterator():
            # return windows of size 100 where the first/last 25 frames are from the previous/next batch
            # the first and last window must be padded by copies of the first and last frame of the video
            no_padded_frames_start = 25
            no_padded_frames_end = 25 + 50 - (len(frames) % 50 if len(frames) % 50 != 0 else 50)  # 25 - 74

            start_frame = np.expand_dims(frames[0], 0)
            end_frame = np.expand_dims(frames[-1], 0)
            padded_inputs = np.concatenate(
                [start_frame] * no_padded_frames_start + [frames] + [end_frame] * no_padded_frames_end, 0
            )

            ptr = 0
            while ptr + 100 <= len(padded_inputs):
                out = padded_inputs[ptr:ptr + 100]
                ptr += 50
                yield out[np.newaxis]

        predictions = []

        for inp in input_iterator():
            single_frame_pred, all_frames_pred = self.predict_raw(inp)
            predictions.append((single_frame_pred.numpy()[0, 25:75, 0],
                                all_frames_pred.numpy()[0, 25:75, 0]))

            print("\r[TransNetV2] Processing video frames {}/{}".format(
                min(len(predictions) * 50, len(frames)), len(frames)
            ), end="")
        print("")

        single_frame_pred = np.concatenate([single_ for single_, all_ in predictions])
        all_frames_pred = np.concatenate([all_ for single_, all_ in predictions])

        return single_frame_pred[:len(frames)], all_frames_pred[:len(frames)]  # remove extra padded frames

    def predict_video(self, video_fn: str):
        try:
            import ffmpeg
        except ModuleNotFoundError:
            raise ModuleNotFoundError("For `predict_video` function `ffmpeg` needs to be installed in order to extract "
                                      "individual frames from video file. Install `ffmpeg` command line tool and then "
                                      "install python wrapper by `pip install ffmpeg-python`.")

        print("[TransNetV2] Extracting frames from {}".format(video_fn))
        video_stream, err = ffmpeg.input(video_fn).output(
            "pipe:", format="rawvideo", pix_fmt="rgb24", s="48x27"
        ).run(capture_stdout=True, capture_stderr=True)

        video = np.frombuffer(video_stream, np.uint8).reshape([-1, 27, 48, 3])
        return (video, *self.predict_frames(video))

    @staticmethod
    def predictions_to_scenes(predictions: np.ndarray, threshold: float = 0.5):
        predictions = (predictions > threshold).astype(np.uint8)

        scenes = []
        t, t_prev, start = -1, 0, 0
        for i, t in enumerate(predictions):
            if t_prev == 1 and t == 0:
                start = i
            if t_prev == 0 and t == 1 and i != 0:
                scenes.append([start, i])
            t_prev = t
        if t == 0:
            scenes.append([start, i])

        # just fix if all predictions are 1
        if len(scenes) == 0:
            return np.array([[0, len(predictions) - 1]], dtype=np.int32)

        return np.array(scenes, dtype=np.int32)

    @staticmethod
    def visualize_predictions(frames: np.ndarray, predictions):
        from PIL import Image, ImageDraw

        if isinstance(predictions, np.ndarray):
            predictions = [predictions]

        ih, iw, ic = frames.shape[1:]
        width = 25

        # pad frames so that length of the video is divisible by width
        # pad frames also by len(predictions) pixels in width in order to show predictions
        pad_with = width - len(frames) % width if len(frames) % width != 0 else 0
        frames = np.pad(frames, [(0, pad_with), (0, 1), (0, len(predictions)), (0, 0)])

        predictions = [np.pad(x, (0, pad_with)) for x in predictions]
        height = len(frames) // width

        img = frames.reshape([height, width, ih + 1, iw + len(predictions), ic])
        img = np.concatenate(np.split(
            np.concatenate(np.split(img, height), axis=2)[0], width
        ), axis=2)[0, :-1]

        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)

        # iterate over all frames
        for i, pred in enumerate(zip(*predictions)):
            x, y = i % width, i // width
            x, y = x * (iw + len(predictions)) + iw, y * (ih + 1) + ih - 1

            # we can visualize multiple predictions per single frame
            for j, p in enumerate(pred):
                color = [0, 0, 0]
                color[(j + 1) % 3] = 255

                value = round(p * (ih - 1))
                if value != 0:
                    draw.line((x + j, y, x + j, y - value), fill=tuple(color), width=1)
        return img


import cv2
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False
import glob
import cv2
import os

from sklearn.cluster import KMeans, spectral_clustering
from sklearn.metrics import silhouette_score, pairwise_distances_argmin_min
from scipy.signal import find_peaks

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

import tensorflow as tf
import shutil
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__) #<<<<<<<<<<<<<<<<<<<<
import sys

def sort_frames(list_of_frames, video_id):
    return sorted(list_of_frames, key=lambda x: int(x.split(f'data/frames/{video_id}/')[1].split('.jpg')[0]))

def get_image_features(frames_filepaths):
  image_features = []
  model = VGG16(weights='imagenet', include_top=False)
  for img_path in frames_filepaths:
    print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    vgg16_feature = model.predict(img_data)
    image_features.append(np.array(vgg16_feature).flatten())
  return image_features

def kmeans(image_features, n_clusters):
  silhouette = np.zeros(n_clusters)
  for k in range(2, n_clusters):
    kmeans = KMeans(n_clusters=k).fit_predict(image_features)
    silhouette[k] = silhouette_score(image_features, kmeans)
    print(f'{k} cluster complete')
  return silhouette

def random_frame_idxs_for_cluster(kmeans_labels):
  idxs = []
  for label in range(len(np.unique(kmeans_labels))):
    idxs.append(np.random.choice(np.argwhere(kmeans_labels == label).flatten(), 1)[0])
  return np.sort(np.array(idxs))

def closest_to_centroid_frames(kmeans, image_list, image_features):
  closest, _ = pairwise_distances_argmin_min(
      kmeans.cluster_centers_, image_features)
  return np.array(image_list)[closest]

def create_video_from_frames(frames, title, fps=30, fourcc=cv2.VideoWriter_fourcc('M','J','P','G')):
    img_array = []
    for filename in frames:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

    out = cv2.VideoWriter(title, fourcc, 30, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()



CATEGORIES = ["Biking"]#, "JumpingJack", "Kayaking", "PlayingGuitar", "Rowing", "Skijet", "TaiChi"]
#, "JumpingJack", "Kayaking", "PlayingGuitar", "Rowing", "Skijet", "TaiChi"]#, "handwaving"]#, "jogging", "running", 
   # "walking"]

if __name__ == "__main__":

    # Create directory to store extracted pixel features.
    os.makedirs("data", exist_ok=True)

    
    n_processed_files = 0
    features = []

    for category in CATEGORIES:
        print("Processing category %s" % category)
        cou = 0
        # Get all files in current category's folder.
        folder_path = os.path.join("..", "videos", category)
        filenames = os.listdir(folder_path)

        # List to store features. features[i] stores features for the i-th video
        # in current category.
        #features = []

        for filename in filenames:
            filepath = os.path.join("..", "videos", category, filename)
            path = "/content/PCA-CBVR/DB/UCF101/videos/" + category + "/" + filename
            video_id = category
            cou = cou + 1
            video = filepath
            video_capture = cv2.VideoCapture(video)
            currentframe = 0
            try: 
              if not os.path.exists(f'data/frames/{video_id}'): 
                os.makedirs(f'data/frames/{video_id}') 
            except OSError: 
              print ('Error: Creating directory of data')
            while(True): 
              ret, frame = video_capture.read() 
              if ret: 
                name = f'data/frames/{video_id}/' + str(currentframe) + '.jpg'
                print ('Creating...' + name) 
                cv2.imwrite(name, frame) 
                currentframe += 1
              else: 
                break
            video_capture.release() 
            cv2.destroyAllWindows()

            #VIDEO SEGMENTATION DONE USING PRETRAINED TransnetV2
            model = TransNetV2() 


            video_frames, single_frame_predictions, all_frame_predictions = model.predict_video(video)

            predictions = np.stack([single_frame_predictions, all_frame_predictions], 1)
            #np.savetxt(file + ".predictions.txt", predictions, fmt="%.6f")

            scenes = model.predictions_to_scenes(single_frame_predictions)
            print("number of rows of shot prediction")
            print(len(scenes))
            print(scenes[0])

            if len(scenes) == 1:

              sbd_frames = []
              dir = "/content/PCA-CBVR/DB/UCF101/videos/data/frames/" + category
              k = (16//(len(scenes)))
              print(k)
              k1 = (16 % (len(scenes)))
              print(k1)
              for i in range(0,len(scenes)):
                for j in range(0,k):
              #print(scenes[i][1])

                  sbd_frames.append(dir + "/" + str(scenes[i][0]+j) + ".jpg")
                #sbd_frames.append(dir + "/" + str(scenes[i][1]) + ".jpg")
                
              if k1 > 0:  
                for j in range(0, k1):
                  sbd_frames.append(dir + "/" + str(scenes[len(scenes)-1][0]+j) + ".jpg")

              #print(sbd_frames)

              try: 
                if not os.path.exists(f'dataset/new/{video_id}'): 
                  os.makedirs(f'dataset/new/{video_id}')
                    
                
              except OSError: 
                print ('Error: Creating directory of data')
              create_video_from_frames(sbd_frames, '/content/PCA-CBVR/DB/UCF101/videos/dataset/new/'+ video_id + '/' + filename.replace('.avi','') +'.avi')


            if len(scenes) < 8 and len(scenes) > 1:

              sbd_frames = []
              dir = "/content/PCA-CBVR/DB/UCF101/videos/data/frames/" + category
              k = (16//(len(scenes)-1))
              print(k)
              k1 = (16 % (len(scenes)-1))
              print(k1)
              for i in range(0,len(scenes)-1):
                for j in range(0,k):
              #print(scenes[i][1])

                  sbd_frames.append(dir + "/" + str(scenes[i][0]+j) + ".jpg")
                #sbd_frames.append(dir + "/" + str(scenes[i][1]) + ".jpg")
                
              if k1 > 0:  
                for j in range(0, k1):
                  sbd_frames.append(dir + "/" + str(scenes[len(scenes)-2][0]+j) + ".jpg")

              #print(sbd_frames)

              try: 
                if not os.path.exists(f'dataset/new/{video_id}'): 
                  os.makedirs(f'dataset/new/{video_id}')
                    
                
              except OSError: 
                print ('Error: Creating directory of data')
              create_video_from_frames(sbd_frames, '/content/PCA-CBVR/DB/UCF101/videos/dataset/new/'+ video_id + '/' + filename.replace('.avi','') +'.avi')



            if len(scenes) >= 8:
              sbd_frames = []
              dir = "/content/PCA-CBVR/DB/UCF101/videos/data/frames/" + category
              for i in range(0,len(scenes)):
                #print(scenes[i][1])

                sbd_frames.append(dir + "/" + str(scenes[i][0]) + ".jpg")
                sbd_frames.append(dir + "/" + str(scenes[i][1]) + ".jpg")

                
              ec_images = sbd_frames
              images_features = get_image_features(ec_images)
                # Dimensionality reduction required before kmeans application
                # finding out optimal number of clusters by implementing a low time complexity algorithm and assign it to n_clusters
              kmeans5 = KMeans(n_clusters=16, random_state=42).fit(images_features)
              five_frames = closest_to_centroid_frames(kmeans5, ec_images, images_features)
              sorted_five_frames = sort_frames(five_frames, category)
              original_frames = sort_frames(sorted_five_frames, category)
              
              print("Shot boundary detection + clustering")
              print(original_frames)

              try: 
                if not os.path.exists(f'dataset/new/{video_id}'): 
                  os.makedirs(f'dataset/new/{video_id}')
                    
                
              except OSError: 
                print ('Error: Creating directory of data')
              create_video_from_frames(original_frames, '/content/PCA-CBVR/DB/UCF101/videos/dataset/new/'+ video_id + '/' + filename.replace('.avi','') +'.avi')




                #sbd_frames = np.array(sbd_frames)
                #print(sbd_frames)
                
                #th = 0
                  #for i in range(0,len(sbd_frames)):
            #if not os.path.exists(sbd_frames[len(sbd_frames)-1]):
            #  sbd_frames[len(sbd_frames)-1] = dir + "/" + str(int(scenes[i][1])) + ".jpg"



            #sbd_frames = np.array(sbd_frames)
            #print("length of 16 frames")
            #print(len(sbd_frames))


            
            shutil.rmtree(f'data/frames/{video_id}')

            
            #shutil.rmtree(f'/content/KTH-Action-Recognition/dataset/data/frames/{video_id}')


            n_processed_files += 1
            if n_processed_files % 30 == 0:
                print("Done %d files" % n_processed_files)

        # Dump data to file.
#pickle.dump(features, open("data/pixel.p" , "wb"))




Processing category Biking
Creating...data/frames/Biking/0.jpg
Creating...data/frames/Biking/1.jpg
Creating...data/frames/Biking/2.jpg
Creating...data/frames/Biking/3.jpg
Creating...data/frames/Biking/4.jpg
Creating...data/frames/Biking/5.jpg
Creating...data/frames/Biking/6.jpg
Creating...data/frames/Biking/7.jpg
Creating...data/frames/Biking/8.jpg
Creating...data/frames/Biking/9.jpg
Creating...data/frames/Biking/10.jpg
Creating...data/frames/Biking/11.jpg
Creating...data/frames/Biking/12.jpg
Creating...data/frames/Biking/13.jpg
Creating...data/frames/Biking/14.jpg
Creating...data/frames/Biking/15.jpg
Creating...data/frames/Biking/16.jpg
Creating...data/frames/Biking/17.jpg
Creating...data/frames/Biking/18.jpg
Creating...data/frames/Biking/19.jpg
Creating...data/frames/Biking/20.jpg
Creating...data/frames/Biking/21.jpg
Creating...data/frames/Biking/22.jpg
Creating...data/frames/Biking/23.jpg
Creating...data/frames/Biking/24.jpg
Creating...data/frames/Biking/25.jpg
Creating...data/frame

In [35]:
shutil.rmtree('/content/PCA-CBVR/DB/UCF101/videos/dataset')
shutil.rmtree('/content/PCA-CBVR/DB/UCF101/videos/data')

In [34]:
scenes

array([[  0, 247]], dtype=int32)

In [36]:
sbd_frames

[]

In [ ]:
sbd_frames = []
            dir = "/content/PCA-CBVR/DB/UCF101/videos/data/frames/" + category
            for i in range(0,len(scenes)):
              #print(scenes[i][1])

              sbd_frames.append(dir + "/" + str(scenes[i][0]) + ".jpg")
              sbd_frames.append(dir + "/" + str(scenes[i][1]) + ".jpg")


            #sbd_frames = np.array(sbd_frames)
            print(sbd_frames)
            
            th = 4
            #for i in range(0,len(sbd_frames)):
            if not os.path.exists(sbd_frames[len(sbd_frames)-1]):
              sbd_frames[len(sbd_frames)-1] = dir + "/" + str(int(scenes[i][1]) - th) + ".jpg"




            ec_images = sbd_frames
            images_features = get_image_features(ec_images)
            # Dimensionality reduction required before kmeans application
            # finding out optimal number of clusters by implementing a low time complexity algorithm and assign it to n_clusters
            kmeans5 = KMeans(n_clusters=2, random_state=42).fit(images_features)
            five_frames = closest_to_centroid_frames(kmeans5, ec_images, images_features)
            sorted_five_frames = sort_frames(five_frames, category)
            original_frames = sort_frames(sorted_five_frames, category)
            
            print("Shot boundary detection + clustering")
            print(original_frames)
            
            #original_frames = sbd_frames
            try: 
              if not os.path.exists(f'dataset/new/{video_id}'): 
                os.makedirs(f'dataset/new/{video_id}')
                 
                
            except OSError: 
              print ('Error: Creating directory of data')
            create_video_from_frames(original_frames, '/content/PCA-CBVR/DB/UCF101/videos/dataset/new/'+ video_id + '/' + filename +'.mp4')


In [ ]:
#TO BE USED FOR EXECUTION
#CODE FOR EXTRACTING PIXEL VALUES as features and storing as feature files all categories in one pickle file
#Shot Boundary Detection and Clustering for getting summarized video

import os
import numpy as np
import tensorflow as tf

__file__ = "/content/KTH-Action-Recognition/dataset/TransNetV2/inference/"
class TransNetV2:

    def __init__(self, model_dir=None):
        if model_dir is None:
            model_dir = os.path.join(os.path.dirname(__file__), "transnetv2-weights/")
            if not os.path.isdir(model_dir):
                raise FileNotFoundError(f"[TransNetV2] ERROR: {model_dir} is not a directory.")
            else:
                print(f"[TransNetV2] Using weights from {model_dir}.")

        self._input_size = (27, 48, 3)
        try:
            self._model = tf.saved_model.load(model_dir)
        except OSError as exc:
            raise IOError(f"[TransNetV2] It seems that files in {model_dir} are corrupted or missing. "
                          f"Re-download them manually and retry. For more info, see: "
                          f"https://github.com/soCzech/TransNetV2/issues/1#issuecomment-647357796") from exc

    def predict_raw(self, frames: np.ndarray):
        assert len(frames.shape) == 5 and frames.shape[2:] == self._input_size, \
            "[TransNetV2] Input shape must be [batch, frames, height, width, 3]."
        frames = tf.cast(frames, tf.float32)

        logits, dict_ = self._model(frames)
        single_frame_pred = tf.sigmoid(logits)
        all_frames_pred = tf.sigmoid(dict_["many_hot"])

        return single_frame_pred, all_frames_pred

    def predict_frames(self, frames: np.ndarray):
        assert len(frames.shape) == 4 and frames.shape[1:] == self._input_size, \
            "[TransNetV2] Input shape must be [frames, height, width, 3]."

        def input_iterator():
            # return windows of size 100 where the first/last 25 frames are from the previous/next batch
            # the first and last window must be padded by copies of the first and last frame of the video
            no_padded_frames_start = 25
            no_padded_frames_end = 25 + 50 - (len(frames) % 50 if len(frames) % 50 != 0 else 50)  # 25 - 74

            start_frame = np.expand_dims(frames[0], 0)
            end_frame = np.expand_dims(frames[-1], 0)
            padded_inputs = np.concatenate(
                [start_frame] * no_padded_frames_start + [frames] + [end_frame] * no_padded_frames_end, 0
            )

            ptr = 0
            while ptr + 100 <= len(padded_inputs):
                out = padded_inputs[ptr:ptr + 100]
                ptr += 50
                yield out[np.newaxis]

        predictions = []

        for inp in input_iterator():
            single_frame_pred, all_frames_pred = self.predict_raw(inp)
            predictions.append((single_frame_pred.numpy()[0, 25:75, 0],
                                all_frames_pred.numpy()[0, 25:75, 0]))

            print("\r[TransNetV2] Processing video frames {}/{}".format(
                min(len(predictions) * 50, len(frames)), len(frames)
            ), end="")
        print("")

        single_frame_pred = np.concatenate([single_ for single_, all_ in predictions])
        all_frames_pred = np.concatenate([all_ for single_, all_ in predictions])

        return single_frame_pred[:len(frames)], all_frames_pred[:len(frames)]  # remove extra padded frames

    def predict_video(self, video_fn: str):
        try:
            import ffmpeg
        except ModuleNotFoundError:
            raise ModuleNotFoundError("For `predict_video` function `ffmpeg` needs to be installed in order to extract "
                                      "individual frames from video file. Install `ffmpeg` command line tool and then "
                                      "install python wrapper by `pip install ffmpeg-python`.")

        print("[TransNetV2] Extracting frames from {}".format(video_fn))
        video_stream, err = ffmpeg.input(video_fn).output(
            "pipe:", format="rawvideo", pix_fmt="rgb24", s="48x27"
        ).run(capture_stdout=True, capture_stderr=True)

        video = np.frombuffer(video_stream, np.uint8).reshape([-1, 27, 48, 3])
        return (video, *self.predict_frames(video))

    @staticmethod
    def predictions_to_scenes(predictions: np.ndarray, threshold: float = 0.5):
        predictions = (predictions > threshold).astype(np.uint8)

        scenes = []
        t, t_prev, start = -1, 0, 0
        for i, t in enumerate(predictions):
            if t_prev == 1 and t == 0:
                start = i
            if t_prev == 0 and t == 1 and i != 0:
                scenes.append([start, i])
            t_prev = t
        if t == 0:
            scenes.append([start, i])

        # just fix if all predictions are 1
        if len(scenes) == 0:
            return np.array([[0, len(predictions) - 1]], dtype=np.int32)

        return np.array(scenes, dtype=np.int32)

    @staticmethod
    def visualize_predictions(frames: np.ndarray, predictions):
        from PIL import Image, ImageDraw

        if isinstance(predictions, np.ndarray):
            predictions = [predictions]

        ih, iw, ic = frames.shape[1:]
        width = 25

        # pad frames so that length of the video is divisible by width
        # pad frames also by len(predictions) pixels in width in order to show predictions
        pad_with = width - len(frames) % width if len(frames) % width != 0 else 0
        frames = np.pad(frames, [(0, pad_with), (0, 1), (0, len(predictions)), (0, 0)])

        predictions = [np.pad(x, (0, pad_with)) for x in predictions]
        height = len(frames) // width

        img = frames.reshape([height, width, ih + 1, iw + len(predictions), ic])
        img = np.concatenate(np.split(
            np.concatenate(np.split(img, height), axis=2)[0], width
        ), axis=2)[0, :-1]

        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)

        # iterate over all frames
        for i, pred in enumerate(zip(*predictions)):
            x, y = i % width, i // width
            x, y = x * (iw + len(predictions)) + iw, y * (ih + 1) + ih - 1

            # we can visualize multiple predictions per single frame
            for j, p in enumerate(pred):
                color = [0, 0, 0]
                color[(j + 1) % 3] = 255

                value = round(p * (ih - 1))
                if value != 0:
                    draw.line((x + j, y, x + j, y - value), fill=tuple(color), width=1)
        return img


import cv2
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False
import glob
import cv2
import os

from sklearn.cluster import KMeans, spectral_clustering
from sklearn.metrics import silhouette_score, pairwise_distances_argmin_min
from scipy.signal import find_peaks

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

import tensorflow as tf
import shutil
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__) #<<<<<<<<<<<<<<<<<<<<
import sys

def sort_frames(list_of_frames, video_id):
    return sorted(list_of_frames, key=lambda x: int(x.split(f'data/frames/{video_id}/')[1].split('.jpg')[0]))

def get_image_features(frames_filepaths):
  image_features = []
  model = VGG16(weights='imagenet', include_top=False)
  for img_path in frames_filepaths:
    print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    vgg16_feature = model.predict(img_data)
    image_features.append(np.array(vgg16_feature).flatten())
  return image_features

def kmeans(image_features, n_clusters):
  silhouette = np.zeros(n_clusters)
  for k in range(2, n_clusters):
    kmeans = KMeans(n_clusters=k).fit_predict(image_features)
    silhouette[k] = silhouette_score(image_features, kmeans)
    print(f'{k} cluster complete')
  return silhouette

def random_frame_idxs_for_cluster(kmeans_labels):
  idxs = []
  for label in range(len(np.unique(kmeans_labels))):
    idxs.append(np.random.choice(np.argwhere(kmeans_labels == label).flatten(), 1)[0])
  return np.sort(np.array(idxs))

def closest_to_centroid_frames(kmeans, image_list, image_features):
  closest, _ = pairwise_distances_argmin_min(
      kmeans.cluster_centers_, image_features)
  return np.array(image_list)[closest]

def create_video_from_frames(frames, title, fps=30, fourcc=cv2.VideoWriter_fourcc('m','p','4','v')):
    img_array = []
    for filename in frames:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

    out = cv2.VideoWriter(title, fourcc, 30, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()



CATEGORIES = ["JumpingJack2"]#, "JumpingJack", "Kayaking", "PlayingGuitar", "Rowing", "Skijet", "TaiChi"]#, "handwaving"]#, "jogging", "running", 
   # "walking"]

if __name__ == "__main__":

    # Create directory to store extracted pixel features.
    os.makedirs("data", exist_ok=True)

    
    n_processed_files = 0
    features = []

    for category in CATEGORIES:
        print("Processing category %s" % category)
        cou = 0
        # Get all files in current category's folder.
        folder_path = os.path.join("..", "dataset", category)
        filenames = os.listdir(folder_path)

        # List to store features. features[i] stores features for the i-th video
        # in current category.
        #features = []

        for filename in filenames:
            filepath = os.path.join("..", "dataset", category, filename)
            path = "/content/KTH-Action-Recognition/dataset/" + category + "/" + filename
            video_id = category
            cou = cou + 1
            video = filepath
            video_capture = cv2.VideoCapture(video)
            currentframe = 0
            try: 
              if not os.path.exists(f'data/frames/{video_id}'): 
                os.makedirs(f'data/frames/{video_id}') 
            except OSError: 
              print ('Error: Creating directory of data')
            while(True): 
              ret, frame = video_capture.read() 
              if ret: 
                name = f'data/frames/{video_id}/' + str(currentframe) + '.jpg'
                print ('Creating...' + name) 
                cv2.imwrite(name, frame) 
                currentframe += 1
              else: 
                break
            video_capture.release() 
            cv2.destroyAllWindows()

            #VIDEO SEGMENTATION DONE USING PRETRAINED TransnetV2
            model = TransNetV2() 


            video_frames, single_frame_predictions, all_frame_predictions = model.predict_video(video)

            predictions = np.stack([single_frame_predictions, all_frame_predictions], 1)
            #np.savetxt(file + ".predictions.txt", predictions, fmt="%.6f")

            scenes = model.predictions_to_scenes(single_frame_predictions)
            sbd_frames = []
            dir = "/content/KTH-Action-Recognition/dataset/data/frames/" + category
            for i in range(0,len(scenes)):
              #print(scenes[i][1])

              sbd_frames.append(dir + "/" + str(scenes[i][0]) + ".jpg")
              sbd_frames.append(dir + "/" + str(scenes[i][1]) + ".jpg")


            #sbd_frames = np.array(sbd_frames)
            print(sbd_frames)
            
            th = 4
            #for i in range(0,len(sbd_frames)):
            if not os.path.exists(sbd_frames[len(sbd_frames)-1]):
              sbd_frames[len(sbd_frames)-1] = dir + "/" + str(int(scenes[i][1]) - th) + ".jpg"




            ec_images = sbd_frames
            images_features = get_image_features(ec_images)
            # Dimensionality reduction required before kmeans application
            # finding out optimal number of clusters by implementing a low time complexity algorithm and assign it to n_clusters
            kmeans5 = KMeans(n_clusters=2, random_state=42).fit(images_features)
            five_frames = closest_to_centroid_frames(kmeans5, ec_images, images_features)
            sorted_five_frames = sort_frames(five_frames, category)
            original_frames = sort_frames(sorted_five_frames, category)
            
            print("Shot boundary detection + clustering")
            print(original_frames)
            
            #original_frames = sbd_frames
            try: 
              if not os.path.exists(f'dataset/new/{video_id}'): 
                os.makedirs(f'dataset/new/{video_id}')
                 
                
            except OSError: 
              print ('Error: Creating directory of data')
            create_video_from_frames(original_frames, '/content/KTH-Action-Recognition/dataset/dataset/new/'+ video_id + '/' + filename +'.mp4')

            shutil.rmtree(f'data/frames/{video_id}')

            
            #shutil.rmtree(f'/content/KTH-Action-Recognition/dataset/data/frames/{video_id}')


            n_processed_files += 1
            if n_processed_files % 30 == 0:
                print("Done %d files" % n_processed_files)

        # Dump data to file.
#pickle.dump(features, open("data/pixel.p" , "wb"))




In [ ]:
import cv2
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False
import glob
import cv2
import os
import cv2
import numpy as np
import json
import time
import math
import os
import shutil
import math
from sklearn.cluster import KMeans
from sklearn import preprocessing


from glob import glob
from matplotlib import pyplot as plt
from collections import Counter
from sklearn.metrics.pairwise import euclidean_distances
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
from torchvision import transforms

CATEGORIES = ["Biking", "JumpingJack", "Kayaking", "PlayingGuitar", "Rowing", "Skijet", "TaiChi"]#, "handwaving"]#, "jogging", "running", 
   # "walking"]

if __name__ == "__main__":
    
    for category in CATEGORIES:
      print("Processing category %s" % category)
      cou = 0
      # Get all files in current category's folder.
      #folder_path = os.path.join("..", "videos", category)
      folder_path = "/content/PCA-CBVR/DB/UCF101/videos/" + category
      filenames = os.listdir(folder_path)
      print(filenames)

    

Processing category Biking
['v_Biking_g15_c03.avi', 'v_Biking_g21_c06.avi', 'v_Biking_g14_c03.avi', 'v_Biking_g24_c06.avi', 'v_Biking_g05_c01.avi', 'v_Biking_g01_c03.avi', 'v_Biking_g16_c01.avi', 'v_Biking_g10_c02.avi', 'v_Biking_g12_c01.avi', 'v_Biking_g08_c05.avi', 'v_Biking_g18_c06.avi', 'v_Biking_g19_c01.avi', 'v_Biking_g16_c03.avi', 'v_Biking_g22_c03.avi', 'v_Biking_g18_c03.avi', 'v_Biking_g21_c04.avi', 'v_Biking_g08_c04.avi', 'v_Biking_g10_c03.avi', 'v_Biking_g13_c06.avi', 'v_Biking_g05_c05.avi', 'v_Biking_g01_c02.avi', 'v_Biking_g13_c02.avi', 'v_Biking_g17_c04.avi', 'v_Biking_g09_c04.avi', 'v_Biking_g15_c01.avi', 'v_Biking_g05_c07.avi', 'v_Biking_g16_c04.avi', 'v_Biking_g03_c02.avi', 'v_Biking_g19_c02.avi', 'v_Biking_g20_c02.avi', 'v_Biking_g22_c01.avi', 'v_Biking_g14_c01.avi', 'v_Biking_g16_c02.avi', 'v_Biking_g11_c04.avi', 'v_Biking_g07_c06.avi', 'v_Biking_g24_c02.avi', 'v_Biking_g25_c01.avi', 'v_Biking_g03_c04.avi', 'v_Biking_g14_c02.avi', 'v_Biking_g08_c03.avi', 'v_Biking_g2

In [ ]:
import shutil
shutil.rmtree("/content/PCA-CBVR/DB/UCF101/frames")

In [ ]:
%cd /content/PCA-CBVR/DB/UCF101/videos

/content/PCA-CBVR/DB/UCF101/videos


In [ ]:
#for installing rar file
!pip install pyunpack
!pip install patool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 2.5 MB/s 


In [ ]:
from pyunpack import Archive
Archive('Biking.rar').extractall('/content/PCA-CBVR/DB/UCF101/videos')
Archive('JumpingJack.rar').extractall('/content/PCA-CBVR/DB/UCF101/videos')
Archive('Kayaking.rar').extractall('/content/PCA-CBVR/DB/UCF101/videos')
Archive('PlayingGuitar.rar').extractall('/content/PCA-CBVR/DB/UCF101/videos')
Archive('Rowing.rar').extractall('/content/PCA-CBVR/DB/UCF101/videos')
Archive('Skijet.rar').extractall('/content/PCA-CBVR/DB/UCF101/videos')
Archive('TaiChi.rar').extractall('/content/PCA-CBVR/DB/UCF101/videos')

!rm *.rar

In [ ]:
# zipping files
!zip -r /content/R2Plus1D50.zip /content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50

  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50/ (stored 0%)
  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50/query/ (stored 0%)
  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50/query/Kayaking/ (stored 0%)
  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50/query/Kayaking/v_Kayaking_g06_c05.npy (deflated 10%)
  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50/query/Kayaking/v_Kayaking_g01_c05.npy (deflated 11%)
  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50/query/Kayaking/v_Kayaking_g06_c04.npy (deflated 11%)
  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50/query/Kayaking/v_Kayaking_g06_c08.npy (deflated 11%)
  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2Plus1D50/query/Kayaking/v_Kayaking_g01_c04.npy (deflated 10%)
  adding: content/PCA-CBVR/MetaDB/Categorical/UCF101/WithoutTune/R2